In [126]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
import os
import re
import json
from nltk.stem import PorterStemmer
import string
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk; nltk.download('stopwords')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy



# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dukeisyourdaddy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [127]:
#pip list

In [128]:
covid_label = pd.read_csv('/Users/dukeisyourdaddy/Desktop/covid_label.csv')

In [129]:
#covid_label.head()

In [130]:
#covid_unru = covid_label[x if x==0 in ['Predicted']]

In [131]:
#covid_label.iloc[1]['text']

In [132]:
#covid_label.iloc[0]['Predicted']

In [133]:
#covid_label.iloc[2]['text']

In [134]:
#len(covid_label)

In [135]:
covid_ru = []
covid_unru = []
for i in range(len(covid_label)):
    if covid_label.iloc[i]['Predicted'] == 0:
        covid_unru.append(covid_label.iloc[i]['text'])
    else:
        covid_ru.append(covid_label.iloc[i]['text'])

In [142]:
def get_sorted(list_a):
    tt = TweetTokenizer()
    #covid_ru_1 = tt.tokenize(covid_ru[1])
    hashtag = {}

    for element in list_a:
        token_element = tt.tokenize(element)
        for item in token_element:
            if '#' in item:
                if item not in hashtag:
                    hashtag[item] = 1
                else:
                    hashtag[item] += 1
                    
    hash_value = sum(hashtag.values())

    # Create a list of tuples sorted by index 1 i.e. value field     
    listofTuples = sorted(hashtag.items() , reverse=True, key=lambda x: x[1])
    # Iterate over the sorted sequence
    sorted_result = []
    for elem in listofTuples :
        sorted_result.append((str(elem[0]) +" :" + str(elem[1]) ))
    
    return sorted_result,hash_value


In [145]:
hash_rumor, rumor_value = get_sorted(covid_ru)
hash_unrumor, unru_value = get_sorted(covid_unru)

In [146]:
hash_rumor[0:11] #select 0:11 cuz we won't count #(lazy to remove it)

['#COVID19 :4186',
 '#coronavirus :2398',
 '#Trump :472',
 '#Coronavirus :365',
 '#Covid19 :362',
 '#covid19 :300',
 '#WuhanVirus :276',
 '# :264',
 '#TrumpVirus :222',
 '#CoronavirusPandemic :207',
 '#CoronaVirus :168']

In [147]:
hash_unrumor[0:11]

['#COVID19 :12441',
 '#coronavirus :6056',
 '#covid19 :1665',
 '#Covid19 :1466',
 '#Coronavirus :1192',
 '#Covid_19 :560',
 '#CoronavirusPandemic :558',
 '#WuhanVirus :539',
 '# :539',
 '#Trump :532',
 '#COVIDー19 :522']

In [148]:
def process_tweet(tweet):
    
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'http\S+', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)
            
    tweet_clean = ' '.join(tweets_clean)

    return tweet_clean

In [149]:
process_covid_ru = [process_tweet(t) for t in covid_ru]
process_covid_unru = [process_tweet(t) for t in covid_unru]

In [65]:
def LDA(corpus):
    # Convert to list
    data = corpus

    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    data_words = list(sent_to_words(data))

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    # Define functions for stopwords, bigrams, trigrams and lemmatization
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out

    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=10, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)


    # Print the Keyword in the 10 topics
    pprint(lda_model.print_topics())
    doc_lda = lda_model[corpus]
    
    # Compute Perplexity
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)
    
    return doc_lda

In [56]:
LDA(process_covid_ru)

[(0,
  '0.105*"trump" + 0.049*"respon" + 0.025*"coronaviru" + 0.021*"presid" + '
  '0.017*"crisi" + 0.017*"obama" + 0.017*"brief" + 0.015*"instead" + '
  '0.013*"american" + 0.012*"fight"'),
 (1,
  '0.041*"covid" + 0.037*"coronaviru" + 0.026*"get" + 0.021*"die" + 0.018*"go" '
  '+ 0.013*"take" + 0.012*"peopl" + 0.010*"make" + 0.009*"kill" + '
  '0.009*"think"'),
 (2,
  '0.022*"rank" + 0.020*"tower" + 0.019*"jake" + 0.019*"russian_collus" + '
  '0.006*"paywal" + 0.006*"clip" + 0.003*"ilk" + 0.003*"rupert_murdoch" + '
  '0.003*"hey" + 0.002*"cue"'),
 (3,
  '0.035*"coronaviru" + 0.032*"death" + 0.023*"covid" + 0.020*"say" + '
  '0.019*"test" + 0.015*"case" + 0.015*"trump" + 0.011*"number" + 0.010*"know" '
  '+ 0.008*"flu"'),
 (4,
  '0.030*"prison" + 0.026*"relea" + 0.021*"asthma" + 0.010*"inmat" + '
  '0.007*"chairman" + 0.005*"confin" + 0.005*"sentenc" + '
  '0.005*"conspiraci_theorist" + 0.004*"tiktok" + 0.003*"prosecutor"'),
 (5,
  '0.019*"awak" + 0.014*"congression" + 0.011*"ron_desan

In [66]:
LDA(process_covid_unru)

[(0,
  '0.073*"child" + 0.072*"school" + 0.041*"student" + 0.033*"open" + '
  '0.033*"kid" + 0.023*"univer" + 0.017*"teacher" + 0.017*"resign" + '
  '0.015*"parent" + 0.014*"reopen"'),
 (1,
  '0.033*"extend" + 0.022*"muslim" + 0.020*"nigerian" + 0.019*"coronavirusupd" '
  '+ 0.019*"lift" + 0.016*"speech" + 0.014*"disappoint" + 0.014*"madagascar" + '
  '0.014*"fight" + 0.014*"sir"'),
 (2,
  '0.073*"covid" + 0.028*"get" + 0.022*"go" + 0.017*"die" + 0.016*"take" + '
  '0.014*"peopl" + 0.011*"see" + 0.011*"coronaviru" + 0.011*"know" + '
  '0.010*"care"'),
 (3,
  '0.046*"help" + 0.044*"tesla" + 0.041*"bill" + 0.040*"employe" + 0.032*"due" '
  '+ 0.030*"park" + 0.020*"job" + 0.020*"money" + 0.019*"food" + '
  '0.014*"payment"'),
 (4,
  '0.060*"restaur" + 0.040*"migrant" + 0.018*"plagu" + 0.017*"queen" + '
  '0.011*"injustic" + 0.010*"hundr" + 0.010*"bloodi" + 0.008*"recruit" + '
  '0.007*"tobacco" + 0.007*"left_we"'),
 (5,
  '0.050*"coronaviru" + 0.021*"say" + 0.014*"trump" + 0.010*"make" + 

In [150]:
def get_event_objects_json(directory):
    json_file = []

    for files in os.listdir(directory):
        file = os.path.join(directory, files)
        json_file.append(file)

    #del json_file[0]
    event_objects = []

    for element in json_file:    
        f = open(element,'r')
        information = []

        for element in f:
            content = json.loads(element)
            information.append(content)

        event_objects.append(information)
    
    return event_objects

In [151]:
rumor_json = get_event_objects_json('/Users/dukeisyourdaddy/Desktop/rumour')
unrumor_json = get_event_objects_json('/Users/dukeisyourdaddy/Desktop/nonrumour')

In [10]:
#Userfule information:
#followers_count, following_count, tweet_count, if descrption is {}, verified
#Table 1 - user features

In [11]:
#influence - followers
#Credibility = verified 
#User role = followers / friends
#Age = creat_id data / publish tweets date
#Friends = followees

In [46]:
def get_user_features(json):
    followers = []
    verified = []
    user_role = []
    age = []
    friends = []
    
    for element in json:
        followers.append(element[0]['user']['public_metrics']['followers_count'])
        
        if element[0]['user']['verified']:
            verified.append(1)
        else:
            verified.append(0)
        
        if element[0]['user']['public_metrics']['following_count'] != 0:
            user_role.append(round(element[0]['user']['public_metrics']['followers_count']/element[0]['user']['public_metrics']['following_count']))
        
        creat_id_date = element[0]['user']['created_at'][0:10]
        publish_date = element[0]['created_at'][0:10]
        
        def cal_days(day1, day2):
            
            dt = pd.to_datetime(day1, format='%Y-%m-%d')
            dt1 = pd.to_datetime(day2, format='%Y-%m-%d')

            return (dt1-dt).days
        
        age.append(cal_days(creat_id_date,publish_date))
        
        friends.append(element[0]['user']['public_metrics']['following_count'])

    
    return followers, verified, user_role, age, friends


In [47]:
rumor_followers, rumor_verified, rumor_user_role, rumor_age, rumor_friends = get_user_features(rumor_json)

In [48]:
un_rumor_followers, un_rumor_verified, un_rumor_user_role, un_rumor_age, un_rumor_friends = get_user_features(unrumor_json)

In [52]:
def get_average(list1):
    return sum(list1)/len(list1)

In [67]:
from statistics import median
from statistics import variance

In [68]:
print(f"Average----rumor_followers is {get_average(rumor_followers)}, un_rumor_followers is {get_average(un_rumor_followers)}")
print(f"Average----rumor_user_role is {get_average(rumor_user_role)}, un_rumor_user_role is {get_average(un_rumor_user_role)}")
print(f"Average----rumor_age is {get_average(rumor_age)}, un_rumor_age is {get_average(un_rumor_age)}")
print(f"Average----rumor_friends is {get_average(rumor_friends)}, un_rumor_friends is {get_average(un_rumor_friends)}")


Average----rumor_followers is 4351047.004085481, un_rumor_followers is 5049828.8797527775
Average----rumor_user_role is 62076.030006317116, un_rumor_user_role is 88514.44068066157
Average----rumor_age is 3405.2536140791954, un_rumor_age is 3358.1394148020654
Average----rumor_friends is 11816.711502199874, un_rumor_friends is 6013.3145047723365


In [69]:
print(f"Median----rumor_followers is {median(rumor_followers)}, un_rumor_followers is {median(un_rumor_followers)}")
print(f"Median----rumor_user_role is {median(rumor_user_role)}, un_rumor_user_role is {median(un_rumor_user_role)}")
print(f"Median----rumor_age is {median(rumor_age)}, un_rumor_age is {median(un_rumor_age)}")
print(f"Median----rumor_friends is {median(rumor_friends)}, un_rumor_friends is {median(un_rumor_friends)}")



Median----rumor_followers is 813918.5, un_rumor_followers is 528566.5
Median----rumor_user_role is 361.0, un_rumor_user_role is 359.0
Median----rumor_age is 3941.0, un_rumor_age is 3795.5
Median----rumor_friends is 1588.0, un_rumor_friends is 1100.0


In [70]:
print(f"variance----rumor_followers is {variance(rumor_followers)}, un_rumor_followers is {variance(un_rumor_followers)}")
print(f"variance----rumor_user_role is {variance(rumor_user_role)}, un_rumor_user_role is {variance(un_rumor_user_role)}")
print(f"variance----rumor_age is {variance(rumor_age)}, un_rumor_age is {variance(un_rumor_age)}")
print(f"variance----rumor_friends is {variance(rumor_friends)}, un_rumor_friends is {variance(un_rumor_friends)}")

variance----rumor_followers is 112773249180229.73, un_rumor_followers is 159127604235677.75
variance----rumor_user_role is 92920854619.22722, un_rumor_user_role is 407152536342.7176
variance----rumor_age is 1501080.257885394, un_rumor_age is 1461679.781516534
variance----rumor_friends is 1130494753.5200121, un_rumor_friends is 708211753.3152094


In [111]:
rumor_verified.count(1)

2529

In [112]:
rumor_verified.count(0)

653

In [115]:
round(653/2529, 2)

0.26

In [113]:
un_rumor_verified.count(1)

10121

In [114]:
un_rumor_verified.count(0)

2661

In [116]:
round(2661/10121, 2)

0.26